# Teams and Hotels

Playing around with a backtracking algorithm to solve the problem of assigning teams to hotels during an athletics event. 

In [15]:
import pandas
import numpy as np

In [16]:
teams = pandas.read_csv('C:/Users/jsmulter/Desktop/test.csv', sep=';') \
    .assign(total = lambda df: df['single'] + df['double'])

In [17]:
# TODO: use DataFrame?
hotels = [
    {
        'name': 'Hotel A',
        'capacity_single': 80,
        'capacity_double': 50,
    },
    {
        'name': 'Hotel B',
        'capacity_single': 40,
        'capacity_double': 20,
    },
    {
        'name': 'Hotel C',
        'capacity_single': 40,
        'capacity_double': 20,
    },
    {
        'name': 'Hotel D',
        'capacity_single': 80,
        'capacity_double': 55,
    },
    
]


In [18]:
teams.sort_values(['total'], ascending=False, inplace=True)

In [19]:
from itertools import cycle, islice

def solve(hotels, teams):
    for hotel in hotels:
        hotel['remaining_single'] = hotel['capacity_single']
        hotel['remaining_double'] = hotel['capacity_double']

    teams_hotels = teams.assign(hotel=None).reset_index(drop=True)
    
    if not place_team(hotels, teams_hotels, 0):
        return 'No solution exists.'
    else:
        return teams_hotels
    
def cycle_hotels(hotels, i):
    # - First hotel is tried first (should be filled to capacity)
    # - Round-robin through the other hotels
    cycled_hotels = hotels[:1]
    num_cycled_hotels = len(hotels) - 1
    offset = i % num_cycled_hotels
    cycled_hotels += islice(cycle(hotels[1:]), offset, offset + num_cycled_hotels)
    return cycled_hotels
    
def place_team(hotels, teams_hotels, i):
    if i >= len(teams_hotels):
        # Exit condition -- all teams have been placed
        return True
    
    t = teams_hotels.loc[i]

    cycled_hotels = cycle_hotels(hotels, i)

    for hotel in cycled_hotels:
        if can_place_team(hotel, t):
            teams_hotels.loc[i, 'hotel'] = hotel['name']
            hotel['remaining_single'] -= t['single']
            hotel['remaining_double'] -= t['double']
            
            if place_team(hotels, teams_hotels, i + 1):
                return True
            
            # Backtrack
            teams_hotels.loc[i, 'hotel'] = None
            hotel['remaining_single'] += t['single']
            hotel['remaining_double'] += t['double']
            
    return False

def can_place_team(hotel, team):
    return team['hotel'] is None and \
           hotel['remaining_single'] >= team['single'] and \
           hotel['remaining_double'] >= team['double']


In [20]:
solve(hotels, teams)

,country,single,double,total,hotel
0,KOS,14,45,59,Hotel A
1,FIN,13,40,53,Hotel D
2,SRB,6,12,18,Hotel D
3,SUI,1,9,10,Hotel B
4,SWE,2,6,8,Hotel C
5,LIE,2,3,5,Hotel A


TODO: 
* export teams as csv
* use csv files and pandas data frame for hotels